In [2]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import efficientnet_b4
from torch import nn
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm

# Ayarlar
MODEL_PATH = r"D:\PlantMaster\PlantMaster_Model\modeller\efficientb4_output\efficientb4_best_model.pth"
DATA_ROOT = r"D:\PlantMaster\Dataset\DuzenlenmisPG-tts"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_path = r"D:\PlantMaster\Dataset\test3"

# Veri yükleyici
def load_test_data(data_root, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    test_dataset = datasets.ImageFolder(test_path, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return test_loader, test_dataset.classes

# Modeli yükle
def load_model(num_classes):
    model = efficientnet_b4(pretrained=False)
    num_ftrs = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.4, inplace=True),
        nn.Linear(num_ftrs, num_classes),
    )
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()
    return model

# Test ve metrikler
def evaluate(model, test_loader, class_names):
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing", leave=False):
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    report = classification_report(all_labels, all_preds, target_names=class_names)
    print("\nSınıf Bazlı Metrikler:\n")
    print(report)

# Ana akış
def main():
    test_loader, class_names = load_test_data(DATA_ROOT)
    model = load_model(num_classes=len(class_names))
    evaluate(model, test_loader, class_names)

if __name__ == "__main__":
    main()


c:\Users\Emircan\anaconda3\envs\tfgpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Emircan\anaconda3\envs\tfgpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
                                                      


Sınıf Bazlı Metrikler:

                                                    precision    recall  f1-score   support

                                 Apple__Apple_scab       0.50      0.25      0.33         4
                                  Apple__Black_rot       0.33      0.20      0.25         5
                           Apple__Cedar_apple_rust       0.00      0.00      0.00         4
                                    Apple__healthy       0.80      1.00      0.89         4
                               Blueberry___healthy       0.20      1.00      0.33         1
          Cherry_(including_sour)___Powdery_mildew       0.33      0.50      0.40         4
                 Cherry_(including_sour)___healthy       0.00      0.00      0.00         4
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.25      1.00      0.40         4
                       Corn_(maize)___Common_rust_       1.00      0.25      0.40         4
               Corn_(maize)___Northern_Leaf_Blight    

c:\Users\Emircan\anaconda3\envs\tfgpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Emircan\anaconda3\envs\tfgpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Emircan\anaconda3\envs\tfgpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


^C
Note: you may need to restart the kernel to use updated packages.


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for lazy-object-proxy: filename=lazy_object_proxy-1.4.3-cp310-cp310-win_amd64.whl size=20209 sha256=f369feb0ae229baae05ed75f07b0a7272d94f9e0981513fe154652035be13cb6
  Stored in directory: c:\users\emircan\appdata\local\pip\cache\wheels\86\84\95\b104d41ac3bf2911ab3fa69817a89799753f696d62978c5bf5
  Created wheel for wrapt: filename=wrapt-1.12.1-cp310-cp310-win_amd64.whl size=32962 sha256=8d8325071ce13ef665a65f6c6842d17bdb229d2cec00822e85808f8c17984139
  Stored in directory: c:\users\emircan\appdata\local\pip\cache\wheels\8e\61\d3\d9e7053100177668fa

  DEPRECATION: Building 'wrapt' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wrapt'. Discussion can be found at https://github.com/pypa/pip/issues/6334
